In [1]:
library(sf)
library(tidyverse)
library(gstat)
library(mgcv)

Linking to GEOS 3.9.1, GDAL 3.3.2, PROJ 7.2.1; sf_use_s2() is TRUE

── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: nlme


Attaching package: 'nlme'


The following object is masked from 'package:dplyr':

    collapse


This is mgcv 1.8-40. For overview type 'help("mgcv-package")'.



In [2]:
# load the data
pad_voronoi <- read_sf("pad_voronoi.shp")
germany <- pad_voronoi %>% 
  st_geometry() %>%
  st_union()

pad_mds <- read_csv("pad_mds.csv") %>%
  st_as_sf(coords = c("LONGITUDE", "LATITUDE")) # 183 obs in total
st_crs(pad_mds) <- 4326

pad_grid <- germany %>% 
  st_bbox() %>%
  st_as_sfc() %>%
  st_make_grid(
    cellsize = c(.05, .05),
    what="centers"
  ) %>% 
  st_as_sf(crs=4326)
st_crs(pad_mds) <- 4326

Rows: 183 Columns: 7
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): doculect, col
dbl (5): LONGITUDE, LATITUDE, r, g, b

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
# divide into training and test set
pad_train = sample_n(pad_mds, 165) # 90% of the data for training
st_crs(pad_train) <- 4326

outside <- sapply(st_intersects(pad_mds, pad_train),function(x){length(x)==0})
pad_test = pad_mds[outside, ] # 10% of the data for testing
st_crs(pad_test) <- 4326

In [4]:
# Inverse distance weighted interpolation
pad.idw <- idw(r ~ 1, location = pad_train, newdata = pad_test, idp=2)
pad.idw

[inverse distance weighted interpolation]


Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



,var1.pred,var1.var,geometry
,<dbl>,<dbl>,<POINT [°]>
1,0.7287666,NA,POINT (11.95648 49.07279)
2,0.7711108,NA,POINT (10.61456 49.06389)
3,0.5010620,NA,POINT (8.86801 50.974)
4,0.6668912,NA,POINT (7.63783 49.60773)
5,0.3799918,NA,POINT (9.93846 51.25775)
6,0.5280494,NA,POINT (8.85498 50.76017)
7,0.6696772,NA,POINT (9.09816 49.71315)
8,0.3634762,NA,POINT (8.12967 51.63018)
9,0.3327884,NA,POINT (10.48229 51.5915)


In [5]:
# Ordinary Kriging
pad.v <- pad_train %>% variogram(r ~ 1, ., cloud=F, cutoff=1000) 
myVariogramModel <- vgm(psill=0.28, "Sph", range=1000, nugget=0.02)
pad.vfit <- fit.variogram(pad.v, myVariogramModel, fit.ranges=F)
pad.krige <- krige(r ~ 1, pad_train, pad_test, pad.vfit)
pad.krige

[using ordinary kriging]


,var1.pred,var1.var,geometry
,<dbl>,<dbl>,<POINT [°]>
1,0.82286424,0.008960758,POINT (11.95648 49.07279)
2,0.86152696,0.006145824,POINT (10.61456 49.06389)
3,0.50797537,0.005547854,POINT (8.86801 50.974)
4,0.75203729,0.009845640,POINT (7.63783 49.60773)
5,0.26892743,0.006988151,POINT (9.93846 51.25775)
6,0.56905018,0.005391376,POINT (8.85498 50.76017)
7,0.84495558,0.011671200,POINT (9.09816 49.71315)
8,0.27814189,0.010394476,POINT (8.12967 51.63018)
9,0.31652175,0.003753346,POINT (10.48229 51.5915)


In [6]:
# Generalized Additive Models
gam.fit <- pad_train %>%
  cbind(., st_coordinates(.)) %>%
  select(r, X, Y) %>%
  mgcv::gam(r ~ s(X,Y), data=.) 

gam.prediction <- predict(gam.fit, newdata = data.frame(st_coordinates(pad_test)))

pad.gam <- pad.krige %>%
  cbind(., st_coordinates(.)) %>%
  st_drop_geometry() %>%
  mutate(Z = gam.prediction) %>%
  select(X, Y, Z) 
#   %>% raster::rasterFromXYZ(crs=4326) %>%
#   as("SpatRaster")

pad.gam

,X,Y,Z
,<dbl>,<dbl>,<dbl>
1,11.95648,49.07279,0.7671347
2,10.61456,49.06389,0.8620809
3,8.86801,50.97400,0.4879954
4,7.63783,49.60773,0.7391883
5,9.93846,51.25775,0.3878407
6,8.85498,50.76017,0.5408546
7,9.09816,49.71315,0.7766164
8,8.12967,51.63018,0.3427526
9,10.48229,51.59150,0.2816372


In [8]:
# calculate Root Mean Square Error(RMSE) from three different models
rmse.idw <- sqrt(mean((pad_test$r - pad.idw$var1.pred)^2))
rmse.krige <- sqrt(mean((pad_test$r - pad.krige$var1.pred)^2))
rmse.gam <- sqrt(mean((pad_test$r - pad.gam$Z)^2))

cat("RMSE value of IDW model is ", rmse.idw, 
    "\nRMSE value of Ordinary Kriging model is ", rmse.krige, 
    "\nRMSE value of GAM is ", rmse.gam)

RMSE value of IDW model is  0.1654976 
RMSE value of Ordinary Kriging model is  0.1668489 
RMSE value of GAM is  0.1467285

In [9]:
# compare the RMSE values of each model
sort(c(rmse.gam, rmse.idw, rmse.krige)) # rmse.gam < rmse.krige < rmse.idw

[1] 0.1467285 0.1654976 0.1668489

Conclusion: 
Comparison of the RMSE values shows us that GAM model has the lowest RMSE value.
This means when we use GAM, we can fit the data the best out of the three spatial smoothing models.